# Quick Example of Blind Source Separation
In this notebook, we will show you a quick example of blind source separation by auxiliary-function-based Laplace-IVA.

In [ ]:
!python -m pip install --upgrade setuptools
!pip install git+https://github.com/tky823/ssspy.git

In [ ]:
import numpy as np
import scipy.signal as ss
import IPython.display as ipd
import matplotlib.pyplot as plt

You can download sample speech data by `ssspy.utils.dataset.download_sample_speech_data`.

In [ ]:
from ssspy.utils.dataset import download_sample_speech_data

In [ ]:
n_sources = 3
max_duration = 10
n_fft, hop_length = 4096, 2048
window = "hann"

In [ ]:
waveform_src_img, sample_rate = download_sample_speech_data(
    n_sources=n_sources,
    max_duration=max_duration,
    conv=True,
)  # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

Let's listen to mixtures.

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

`ssspy.bss.iva.AuxLaplaceIVA` provides Auxiliary-function-based Laplace-IVA.

In [ ]:
from ssspy.bss.iva import AuxLaplaceIVA

In [ ]:
iva = AuxLaplaceIVA()
print(iva)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window=window, nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = iva(spectrogram_mix, n_iter=100)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window=window, nperseg=n_fft, noverlap=n_fft-hop_length)

Let's listen to estimated sources.

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

Display cost at each iteration.

In [ ]:
plt.figure()
plt.plot(range(len(iva.loss)), iva.loss)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()
plt.close()